## Search Engine

In [1]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper



In [3]:
api_wrappper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki= WikipediaQueryRun(api_wrapper=api_wrappper_wiki)

In [4]:
wiki.name

'wikipedia'

In [5]:
api_wrappper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrappper_arxiv)


In [6]:
arxiv.name

'arxiv'

In [7]:
tools = [wiki, arxiv]

In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
groq_api_key = os.getenv("GROQ_API_KEY") 

In [10]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-70b-8192")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10e621f90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10e622c50>, model_name='llama3-70b-8192', groq_api_key=SecretStr('**********'))

In [11]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")
os.environ['LANGCHAIN_TRACING_V2'] = "true"

In [13]:
## Prompt Template

from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

In [14]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
from langchain.agents import create_openai_tools_agent

agent= create_openai_tools_agent(model,tools=tools, prompt=prompt)

In [16]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [17]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [19]:
agent_executor.invoke({'input': 'give any AI research paper'})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'AI research paper'}`


Published: 2021-03-29
Title: "Weak AI" is Likely to Never Become "Strong AI", So What is its Greatest Value for us?
Authors: Bin Liu
Summary: AI has surpassed humans across a variety of tasks such as image
classification, playing games (e.g., go, "St
Invoking: `wikipedia` with `{'query': 'Bin Liu'}`


Page: Bin Liu
Summary: Bin Liu is a chemist who is Professor and Provost's Chair at the National University of Singapore. Her research considers polymer chemistry and organic functional materials. She was appointed Vice President of Research and TechThe author of the AI research paper is not the Bin Liu I was expecting. It seems the author is a different person. Let me try to find more information about the author.

> Finished chain.


{'input': 'give any AI research paper',
 'output': 'The author of the AI research paper is not the Bin Liu I was expecting. It seems the author is a different person. Let me try to find more information about the author.'}